In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.externals import joblib
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import KFold
from sklearn import metrics

advertiserlist_path = '../data/adtech_intern_advertiserlist.csv'
log_path = '../data/intern_samplelog.csv'

In [2]:
advertiserlist = pd.read_csv(advertiserlist_path)
log_original = pd.read_csv(log_path)
log = log_original

In [3]:
log.loc[:,'os'][log.os=='iOS'] = 0
log.loc[:,'os'][log.os=='Android'] = 1

/Users/kei/penv2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/kei/penv2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [4]:
def hashnum(modnum, data):
    mod = 10 ** modnum
    return hash(data) % mod

In [23]:
logEachAd = [log[log.advertiser==ad].drop('advertiser', axis=1)for ad in xrange(1, 11)]
input_type='string'
param = {'max_depth':6, 'eta':1, 'silent':1, 'objective':'binary:logistic'}
num_round = 2
modnum = 3
n_folds = 10

xgb_models = [0] * 10
for ind, eachlog in enumerate(logEachAd):
    print '----------------- Advertiser %d -------------------' %(ind+1)
    log_data = eachlog.drop('click', axis=1)
    log_target = eachlog.click.values
    hashlog_data = [[hashnum(modnum, num) for num in sample] for sample in log_data.drop(['floor_price','os'], axis=1).values]
    log_data = np.c_[hashlog_data,log_data.os]
    dlog = xgb.DMatrix(log_data, label=log_target)
    kf = KFold(len(log_target), n_folds=n_folds, shuffle=True, random_state=0)
    for train_index, test_index in kf:
        dlog_train = xgb.DMatrix(
            log_data[train_index], label=log_target[train_index])
        dlog_test = xgb.DMatrix(
            log_data[test_index], label=log_target[test_index])
        xgb_model = xgb.train(param, dlog_train, num_round)
        predict = xgb_model.predict(dlog_test)
        # print metrics.roc_auc_score(log_target[test_index], predict)
    xgb_models[ind] = xgb.train(param, dlog, num_round)

for ind, eachlog in enumerate(logEachAd):
    print '----------------- Advertiser %d -------------------' %(ind+1)
    log_data = eachlog.drop('click', axis=1)
    log_target = eachlog.click.values
    hashlog_data = [[hashnum(modnum, num) for num in sample] for sample in log_data.drop(['floor_price','os'], axis=1).values]
    log_data = np.c_[hashlog_data,log_data.os]
    dlog = xgb.DMatrix(log_data, label=log_target)
    kf = KFold(len(log_target), n_folds=n_folds, shuffle=True, random_state=0)
    for train_index, test_index in kf:
        dlog_train = xgb.DMatrix(
            log_data[train_index], label=log_target[train_index])
        dlog_test = xgb.DMatrix(
            log_data[test_index], label=log_target[test_index])
        newlog_train = np.c_[np.array([model.predict(dlog_train) for model in xgb_models]).T, log_data[train_index]]
        newlog_test = np.c_[np.array([model.predict(dlog_test) for model in xgb_models]).T, log_data[test_index]]
        newdlog_train = xgb.DMatrix(
            newlog_train, label=log_target[train_index])
        newdlog_test = xgb.DMatrix(
            newlog_test, label=log_target[test_index])
        xgb_model = xgb.train(param, newdlog_train, num_round)
        predict = xgb_model.predict(newdlog_test)
        print metrics.roc_auc_score(log_target[test_index], predict)

----------------- Advertiser 1 -------------------
----------------- Advertiser 2 -------------------
----------------- Advertiser 3 -------------------
----------------- Advertiser 4 -------------------
----------------- Advertiser 5 -------------------
----------------- Advertiser 6 -------------------
----------------- Advertiser 7 -------------------
----------------- Advertiser 8 -------------------
----------------- Advertiser 9 -------------------
----------------- Advertiser 10 -------------------
----------------- Advertiser 1 -------------------
0.662212194529
0.65934423551
0.6657092759
0.663910913954
0.667515831428
0.676985007197
0.663298325398
0.660958979252
0.664546792143
0.65849789129
----------------- Advertiser 2 -------------------
0.572032275285
0.575604336996
0.566810732699
0.568758300434
0.577214103771
0.569358303974
0.572279123598
0.570654695308
0.568862045171
0.571465528642
----------------- Advertiser 3 -------------------
0.625256905879
0.62340418157
0.616372515